In [2]:
from scipy.stats import truncnorm
import pandas as pd
import numpy as np
import itertools
import datetime
import tqdm
import sys
import os

def flatten_list(list_array):
    return list(itertools.chain(*list_array))

sys.path.insert(0, "../")
sys.path.insert(0,"../pompjax/pompjax/")

from global_config import config

results_dir           = config.get_property('results_dir')
results2_dir           = config.get_property('results2_dir')

data_dir              = config.get_property('data_dir')
paper_dir             = config.get_property('paper_dir')
data_db_dir           = config.get_property('data_db_dir')
feb_hosp_records_path = os.path.join(data_db_dir, 'long_files_8_25_2021')
path_to_save          = os.path.join(results_dir, "real_testing", "community")

COLOR_LIST1           = ["#F8AFA8", "#FDDDA0", "#F5CDB4", "#74A089"]

In [3]:
from utils_data_metapop import create_population_data, create_time_transfers

path_to_ward_counts = os.path.join(data_db_dir, "long_files_8_25_2021", "counts_ward.csv" )
path_to_ward_transf = os.path.join(data_db_dir, "long_files_8_25_2021", "transfers_ward.csv" )

A_df, D_df, H_df, tests_df, Hmean_df = create_population_data(path_to_ward_counts)

num_wards  = len(Hmean_df)
ward_names = list(Hmean_df.index)
M_df       = create_time_transfers(path_to_ward_transf, num_wards=num_wards, ward_names=ward_names)

# we want to choose synthetic scenarios that overall reproduce the synthetic observations, so we are going to use the stuff above (in the GridSearch) to sample from the parameter space and create the synthetic scenarios randomly.

In [4]:
from scipy.interpolate import UnivariateSpline

def return_score_cutoff(score, cut_off_prob=0.05):
    freq, score = np.histogram(score, bins=100, density=True)
    freq_cum    = np.cumsum(freq); freq_cum = freq_cum/freq_cum[-1]
    score       = score[1:]
    f_cum       = UnivariateSpline(score, freq_cum, s=0.001)
    sc_range    = np.linspace(np.min(score), np.max(score), 1000)
    score_cut   = sc_range[np.argmin(np.abs(f_cum(sc_range) * 100 - cut_off_prob*100))]
    return score_cut


In [5]:
selected_buildings = ['Allen Hospital-Allen', 'Harkness Pavilion-Columbia', 'Milstein Hospital-Columbia', 'Mschony-Chony', 'Presbyterian Hospital-Columbia']
building2id        = {selected_buildings[i]: i for i in range(len(selected_buildings))}

def building2observation(building):
    if building in selected_buildings:
        return building2id[building]
    else:
        return 5

ward_names_df                = pd.DataFrame(ward_names, columns=["ward"])
ward_names_df["building"]    = ward_names_df["ward"].apply(lambda x: "-".join(x.split("-")[1:]))
ward_names_df["buidling_id"] = ward_names_df["building"].apply(lambda x: building2observation(x) )
ward_names_df["ward_id"]     = ward_names_df.apply(lambda x: np.where(ward_names_df.ward == x.ward)[0][0], axis=1)
wardid2buildingid            = {row.ward_id: row.buidling_id for i, row in ward_names_df.iterrows()}


In [6]:
from models import process_metapop, observe_metapop_cluster, init_metapop, simulate_metapop, simulate_metapop_observations
from misc import amro2cute

if_settings = {
   "Nif"                : 20,          # number of iterations of the IF
   "type_cooling"       : "geometric", # type of cooling schedule
   "shrinkage_factor"   : 0.9,         # shrinkage factor for the cooling schedule
   "inflation"          : 1.01,        # inflation factor for spreading the variance after the EAKF step
}

model_settings = {
    "param_name"  : ["ρ", "β"],       # importation and transmission rate
    "p"           : 2,                # number of parameters
    "dt"          : 1,                # time step
    "m"           : 300,              # number of ensembles
    "stochastic"  : True              # is stochastic
    }

dates_simulation = pd.date_range(start=pd.to_datetime("2020-02-01"), end=pd.to_datetime("2021-02-28"), freq="D")
num_pop          = num_wards

model_settings["n"]       = 3*num_pop,             # number of state variables / dimension of the state space
model_settings["k"]       = num_pop,               # number of observations | We are just observing carriage
model_settings["T"]       = len(dates_simulation), # time to run
model_settings["num_pop"] = num_pop,
model_settings["dates"]   = dates_simulation

delta = 1/120  # decolonization rate

A     = A_df.to_numpy()
D     = D_df.to_numpy()
H     = H_df.to_numpy()
M     = M_df
tests = tests_df.to_numpy()

# Process model for the ifeakf | model(x, gamma, beta, delta, rho, sigma, pop, m=1, stochastic=True)
process_model_gamma = lambda t, x, θ, gamma : process_metapop(t, x,
                                            gamma = gamma * np.ones(model_settings["m"]),
                                            beta  = θ[1, :],
                                            delta = delta,
                                            Nmean = np.expand_dims(Hmean_df, -1),
                                            N     = H[:, [t]],
                                            A     = A[:, [t]],
                                            D     = D[:, [t]],
                                            M     = M[:, :, t])

# f0 model for the ifeakf            | initial_condition(c0, pop=2000, m=300)
initial_guess_x0_gamma  = lambda θ, gamma:  init_metapop(
                                                N0             = H[:, 0],
                                                c0             = gamma, # importation rate
                                                model_settings = model_settings)

# Observational model for the ifeakf |  g(t, x, rho)
observational_model  = lambda t, x, θ: observe_metapop_cluster(t, x,
                                                rho            = θ[0, :],
                                                N              = H[:, [t]],
                                                num_tests      = tests[:, [t]],
                                                model_settings = model_settings,
                                                ward2cluster   = wardid2buildingid)

def sample_scenarios(path_to_grid_search, cut_off=10/100):
    gs_df        = pd.read_csv( path_to_grid_search )
    sc_cutoff    = return_score_cutoff(gs_df.crps, cut_off_prob=cut_off)
    gs_df        = gs_df[gs_df.crps <= sc_cutoff].reset_index(drop=True)
    scenarios_df = gs_df.copy()
    scenarios_df = scenarios_df.sample(n=10); scenarios_df = scenarios_df[["rho", "beta", "crps", "calibration_score"]].reset_index(drop=True)
    return scenarios_df

def empirical_prevalence(amro, path_to_prev="../data/amro_prevalence.csv"):
    amro_prev_df = pd.read_csv(path_to_prev)
    gamma        = amro_prev_df[amro_prev_df.amro==amro]["prevalence_mean1"].values[0]/100
    return gamma

# inferences not adjusting the state space, as with the ABM

In [9]:
from diagnostic_plots import convergence_plot
from utils import create_df_response
from ifeakf import ifeakf


def run_amro_synthetic(amro, id_run=0):

    path_to_grid_search = os.path.join(results2_dir, "grid_search", "metapopulation", f"{amro2cute(amro)}.csv")
    scenarios_df        = sample_scenarios(path_to_grid_search, cut_off=10/100)
    gamma               = empirical_prevalence(amro, path_to_prev="../data/amro_prevalence.csv")

    dates_infer  = pd.date_range(start=pd.to_datetime("2020-02-01"), end=pd.to_datetime("2021-02-28"), freq="D")


    path_to_save = os.path.join(results2_dir, "synthetic_inferences", "no_state_space", f"{amro2cute(amro)}")
    os.makedirs(path_to_save, exist_ok=True)

    scenarios_df.to_csv(os.path.join(path_to_save, f"scenarios{id_run}.csv"))

    for idx_row, row in scenarios_df.iterrows():

        model_settings["param_truth"] = [row["rho"], row["beta"]]
        model_settings["num_build"] = len(np.unique(list(wardid2buildingid.values())))
        model_settings["k"]         = model_settings["num_build"] # observing at that aggregation

        process_model        = lambda t, x, θ: process_model_gamma(t, x, θ, gamma=gamma)
        init_conditions      = lambda θ: initial_guess_x0_gamma(θ, gamma=gamma)
        observational_model  = lambda t, x, θ: observe_metapop_cluster(t, x,
                                                    N            = H[:, [t]],
                                                    rho          = θ[0, :],
                                                    num_tests    = tests[:, [t]],
                                                    ward2cluster = wardid2buildingid)

        θtruth       = np.array([model_settings["param_truth"]]).T * np.ones((model_settings["p"], model_settings["m"]))
        x_sim, y_sim = simulate_metapop(
                        process_model       = process_model,
                        observational_model = observational_model,
                        init_state          = init_conditions,
                        θsim                = θtruth,
                        model_settings      = model_settings)

        idx_infer      = np.random.randint(y_sim.shape[1])
        obs_infer      = y_sim[:, :, idx_infer].transpose(1, 0)

        obs_df = pd.DataFrame(index=dates_infer)
        for i in range(model_settings["num_build"]) :
            obs_df['y'+str(i+1)]   = obs_infer[i, :]
            obs_df['oev'+str(i+1)] = 1 +(0.2 * obs_df['y'+str(i+1)].values)**2


        obs_df                  = obs_df.resample("W-Sun").sum()
        obs_df.index.values[-1] = model_settings["dates"][-1]

        ρmin           = 0.01 # test sensitivity minimum
        ρmax           = 0.2  # test sensitivity maximum
        βmin           = 0.00 # transmission rate minimum
        βmax           = 0.5  # transmission rate maximum

        max_total_pop     = np.max(H.sum(axis=0))
        state_space_range = np.array([0, max_total_pop])
        parameters_range  = np.array([[ρmin, ρmax],    [βmin, βmax]])
        σ_perturb         = np.array([(ρmax - ρmin)/4, (βmax - βmin)/4])

        if_settings["assimilation_dates"] = obs_df.index.values
        if_settings["adjust_state_space"] = False  # for comparing with the abm

        path_to_save_sce = os.path.join(results2_dir, "synthetic_inferences", "no_state_space", f"{amro2cute(amro)}", f"scenario{idx_row+1}")
        os.makedirs(path_to_save_sce, exist_ok=True)

        θmle, θpost = ifeakf(process_model                = process_model,
                                observational_model       = observational_model,
                                state_space_initial_guess = init_conditions,
                                observations_df           = obs_df,
                                parameters_range          = parameters_range,
                                state_space_range         = state_space_range,
                                model_settings            = model_settings,
                                if_settings               = if_settings,
                                perturbation              = σ_perturb)

        np.savez_compressed(os.path.join(path_to_save_sce, f"{str(id_run).zfill(3)}posterior.npz"),
                                        mle           = θmle,
                                        posterior     = θpost,
                                        state_space   = x_sim,
                                        observations  = y_sim,
                                        teta_truth    = θtruth,
                                        idx_infer     = idx_infer)


In [10]:
amro_search  = ['ESCHERICHIA COLI', 'KLEBSIELLA PNEUMONIAE',  'PSEUDOMONAS AERUGINOSA',
                'METHICILLIN-SUSCEPTIBLE STAPHYLOCOCCUS AUREUS',
                "STAPHYLOCOCCUS EPIDERMIDIS", 'ENTEROCOCCUS FAECALIS', 'ENTEROCOCCUS FAECIUM']

for amro in amro_search:
    run_amro_synthetic(amro)



In [43]:
from misc import amro2title
import matplotlib.pyplot as plt

amro_search  = ['ESCHERICHIA COLI', 'KLEBSIELLA PNEUMONIAE',  'PSEUDOMONAS AERUGINOSA', 'METHICILLIN-SUSCEPTIBLE STAPHYLOCOCCUS AUREUS',
                "STAPHYLOCOCCUS EPIDERMIDIS", 'ENTEROCOCCUS FAECALIS', 'ENTEROCOCCUS FAECIUM']

#for amro in amro_search:
id_run       = 0
#amro         = amro_search[0]
for amro in amro_search:

    path_to_save = os.path.join(results2_dir, "synthetic_inferences", "no_state_space", f"{amro2cute(amro)}")
    scenarios_df = pd.read_csv(os.path.join(path_to_save, f"scenarios{id_run}.csv")).drop(columns=["Unnamed: 0"])

    fig, ax = plt.subplots(10, 2, figsize=(12, 16.9), sharex=True, sharey="col")
    for idx_row, row  in enumerate(scenarios_df.iterrows()):

        path_to_save_sce = os.path.join(results2_dir, "synthetic_inferences", "no_state_space", f"{amro2cute(amro)}", f"scenario{idx_row+1}")
        inference        = np.load(os.path.join(path_to_save_sce, f"{str(id_run).zfill(3)}posterior.npz"))

        θmle             = inference["mle"]
        θpost            = inference["posterior"]
        x_sim            = inference["state_space"]
        y_sim            = inference["observations"]
        θtruth           = inference["teta_truth"]
        idx_infer        = inference["idx_infer"]

        ρ_df = create_df_response(θpost[0, :, :, :].mean(-2).T, time=if_settings["Nif"])
        β_df = create_df_response(θpost[1, :, :, :].mean(-2).T, time=if_settings["Nif"])

        p_dfs       = [ρ_df, β_df]
        param_label = ["ρ", "β"]

        ρmin = 0.01 # test sensitivity minimum
        ρmax = 0.2  # test sensitivity maximum
        βmin = 0.00 # transmission rate minimum
        βmax = 0.5  # transmission rate maximum
        parameters_range  = np.array([[ρmin, ρmax], [βmin, βmax]])
        convergence_plot(θmle, p_dfs, parameters_range, param_label, ax=ax[idx_row, :], fig=fig, param_truth=list(θtruth[:, 0]))

        ax[idx_row, 0].legend().remove()
        ax[idx_row, 1].legend().remove()
        ax[idx_row, 1].set_xlabel(None)

    ax[-1, 0].set_xlabel("IF iteration")
    ax[-1, 1].set_xlabel("IF iteration")

    fig.suptitle(amro2title(amro), fontsize=14)
    plt.tight_layout()
    fig.savefig(os.path.join(path_to_save, f"convergences{id_run}.png"), dpi=300, bbox_inches="tight", transparent=True)
    plt.close(fig)

# Inferences adjusting the state space

the problem is that pompjax asssume the state space is a vector $x\in R^{n \times m}$, and we putted the number of populations of the meta in the second axis such that $x\in R^{3 \times p \times m }$, and $n=3 \times p$. $p:=$ number of populations.

So we need to change the process model to handle this, I think it could be done easily just adding a np.reshape(-1, m), but we'll see...

In [83]:
from models import binomial_transition, poisson_transition, check_state_space

def process_metapop2(t, x, gamma, beta, delta, Nmean, N, A, D, M, model_settings=None):
    """ Susceptible - Colonized meta-population model

    Args:
        x[t]  : state space
        gamma : importation rate
        beta  : transmission rate
        delta : decolonization rate
        A     : Admitted
        D     : Discharged
        M     : Movements matrix

    Returns:
        x[t+1]: State space in t+1.
    """

    n       = model_settings["n"]
    num_pop = model_settings["num_pop"]
    m       = model_settings["m"]

    x       = np.reshape(x, (int(n/num_pop), num_pop, m))

    C = x[0, :, :]

    S = np.clip(N - C, 0, N)
    with np.errstate(divide='ignore', invalid='ignore'):
        c = np.clip(np.nan_to_num(C/N), 0, 1)

    λ = beta * C / Nmean # force of infection

    # moving out and in colonized
    Cout  = binomial_transition(list(np.sum(M, axis=1, keepdims=True)), c)
    Cin   = M.T @ c

    a2c  = binomial_transition(list(A), gamma) # people admitted colonized.
    c2d  = binomial_transition(list(D), c)     # discharged colonized

    s2c  = poisson_transition(S, λ)     # new colonized
    c2s  = poisson_transition(C, delta) # decolonizations

    C    = C + a2c - c2d + s2c + c2s + Cin - Cout
    C    = np.clip(C, 0, N)

    x    = check_state_space(np.array([C, a2c, s2c]))

    return np.reshape(x, (n, m))

def init_metapop2(N0, c0, model_settings):
    """ Initial conditions model.
        Args:
            N0 (int):    Initial size of populations.
            c0 (int):    Initial fraction of carriers.
        Returns:
            x0 (np.array): Initial conditions of the state space.
    """
    m       = model_settings["m"]
    num_pop = model_settings["num_pop"]
    n       = model_settings["n"]

    N0   = np.expand_dims(N0, -1) * np.ones((num_pop, m))
    C0   = c0 * N0
    AC   = np.zeros((num_pop, m))
    newC = np.zeros((num_pop, m))
    x    = np.array([C0, AC, newC])
    return np.reshape(x, (n, m))

def observe_metapop2(t, x, N, rho, num_tests, model_settings, ward2cluster=None):
    """ Observational model
        Args:
            t (int):      Time
            x (np.array): State space
            rho (float):  Observation probability
        Returns:
            y (np.array): Observed carriers ~ Binomial(C, rho)
    """

    n       = model_settings["n"]
    m       = model_settings["m"]
    num_pop = model_settings["num_pop"]
    k       = model_settings["k"]

    x       = np.reshape(x, (int(n/num_pop), num_pop, m))
    C       = x[0, :, :]

    with np.errstate(divide='ignore', invalid='ignore'):
        c = np.clip(np.nan_to_num(C/N), 0, 1)

    observed_colonized = np.random.binomial(list(num_tests * np.ones((num_pop, m))), rho * c)
    # need to resample this to [num_buildings x m] (maybe using the same buildings that rami used)
    obs_col_building = np.zeros((k, m))

    for i in range(k):
        obs_col_building[ward2cluster[i], :] += observed_colonized[i, :]

    return obs_col_building

# Process model for the ifeakf | model(x, gamma, beta, delta, rho, sigma, pop, m=1, stochastic=True)
process_model_gamma2 = lambda t, x, θ, gamma : process_metapop2(t, x,
                                            gamma = gamma * np.ones(model_settings["m"]),
                                            beta  = θ[1, :],
                                            delta = delta,
                                            Nmean = np.expand_dims(Hmean_df, -1),
                                            N     = H[:, [t]],
                                            A     = A[:, [t]],
                                            D     = D[:, [t]],
                                            M     = M[:, :, t],
                                            model_settings=model_settings)

# f0 model for the ifeakf            | initial_condition(c0, pop=2000, m=300)
initial_guess_x0_gamma2  = lambda θ, gamma:  init_metapop2(
                                                N0             = H[:, 0],
                                                c0             = gamma, # importation rate
                                                model_settings = model_settings)
# Observational model for the ifeakf |  g(t, x, rho)
observational_model      = lambda t, x, θ: observe_metapop2(t, x,
                                                rho            = θ[0, :],
                                                N              = H[:, [t]],
                                                num_tests      = tests[:, [t]],
                                                model_settings = model_settings,
                                                ward2cluster   = wardid2buildingid)

In [86]:
from diagnostic_plots import convergence_plot
from utils import create_df_response
from ifeakf import ifeakf


def run_amro_synthetic2(amro, id_run=0):

    amro_prev_df = pd.read_csv(os.path.join("..", "data", "amro_prevalence.csv"))
    dates_infer  = pd.date_range(start=pd.to_datetime("2020-02-01"), end=pd.to_datetime("2021-02-28"), freq="D")
    gamma        = amro_prev_df[amro_prev_df.amro==amro]["prevalence_mean1"].values[0]/100

    process_model   = lambda t, x, θ : process_model_gamma2(t, x, θ, gamma)
    init_conditions = lambda θ:        initial_guess_x0_gamma2(θ, gamma)

    # use the same scenarios as with the inferences without adjusting the state space
    scenarios_df = pd.read_csv(os.path.join(results2_dir, "synthetic_inferences", "no_state_space",  f"{amro2cute(amro)}", f"scenarios{id_run}.csv")).drop(columns=["Unnamed: 0"])
    path_to_save = os.path.join(results2_dir, "synthetic_inferences", "state_space", f"{amro2cute(amro)}")
    os.makedirs(path_to_save, exist_ok=True)

    #scenarios_df.to_csv(os.path.join(path_to_save, f"scenarios{id_run}.csv"))

    for idx_row, row in scenarios_df.iterrows():

        model_settings["param_truth"] = [row["rho"], row["beta"]]
        model_settings["num_build"] = len(np.unique(list(wardid2buildingid.values())))
        model_settings["k"]         = model_settings["num_build"] # observing at that aggregation

        path_to_save_sce = os.path.join(results2_dir, "synthetic_inferences", "no_state_space", f"{amro2cute(amro)}", f"scenario{idx_row+1}")

        θtruth1          = np.array([model_settings["param_truth"]]).T * np.ones((model_settings["p"], model_settings["m"]))
        inference        = np.load(os.path.join(path_to_save_sce, f"{str(id_run).zfill(3)}posterior.npz"))

        θmle             = inference["mle"]
        θpost            = inference["posterior"]
        x_sim            = inference["state_space"]
        y_sim            = inference["observations"]
        θtruth           = inference["teta_truth"]
        idx_infer        = inference["idx_infer"]

        if not np.array_equal(θtruth1, θtruth):
            print("not equal truths")

        obs_infer      = y_sim[:, :, idx_infer].transpose(1, 0)

        obs_df = pd.DataFrame(index=dates_infer)
        for i in range(model_settings["num_build"]) :
            obs_df['y'+str(i+1)]   = obs_infer[i, :]
            obs_df['oev'+str(i+1)] = 1 +(0.2 * obs_df['y'+str(i+1)].values)**2

        obs_df                  = obs_df.resample("W-Sun").sum()
        obs_df.index.values[-1] = model_settings["dates"][-1]

        ρmin           = 0.01 # test sensitivity minimum
        ρmax           = 0.20 # test sensitivity maximum
        βmin           = 0.00 # transmission rate minimum
        βmax           = 0.50 # transmission rate maximum

        max_total_pop     = np.max(H.sum(axis=0))
        state_space_range = np.array([0, max_total_pop])
        parameters_range  = np.array([[ρmin, ρmax], [βmin, βmax]])
        σ_perturb         = np.array([(ρmax - ρmin)   / 4,
                                        (βmax - βmin) / 4])

        if_settings["assimilation_dates"] = obs_df.index.values
        if_settings["adjust_state_space"] = True

        save_post_path = os.path.join(path_to_save, f"scenario{idx_row+1}")
        os.makedirs(save_post_path, exist_ok=True)

        θmle, θpost = ifeakf(process_model                = process_model,
                                observational_model       = observational_model,
                                state_space_initial_guess = init_conditions,
                                observations_df           = obs_df,
                                parameters_range          = parameters_range,
                                state_space_range         = state_space_range,
                                model_settings            = model_settings,
                                if_settings               = if_settings,
                                perturbation              = σ_perturb)

        np.savez_compressed(os.path.join(save_post_path, f"{str(id_run).zfill(3)}posterior.npz"),
                                        mle           = θmle,
                                        posterior     = θpost,
                                        state_space   = x_sim,
                                        observations  = y_sim,
                                        teta_truth    = θtruth,
                                        idx_infer     = idx_infer)

        ρ_df = create_df_response(θpost[0, :, :, :].mean(-2).T, time=if_settings["Nif"])
        β_df = create_df_response(θpost[1, :, :, :].mean(-2).T, time=if_settings["Nif"])

        p_dfs       = [ρ_df, β_df]
        param_label = ["ρ", "β"]

        parameters_range  = np.array([[ρmin, ρmax], [βmin, βmax]])
        convergence_plot(θmle, p_dfs, parameters_range, param_label, param_truth=list(θtruth[:, 0]),
                            path_to_save=os.path.join(save_post_path, f"{str(id_run).zfill(3)}convergence.png"))

In [87]:
amro_search  = ['ESCHERICHIA COLI', 'KLEBSIELLA PNEUMONIAE',  'PSEUDOMONAS AERUGINOSA',
                'METHICILLIN-SUSCEPTIBLE STAPHYLOCOCCUS AUREUS',
                "STAPHYLOCOCCUS EPIDERMIDIS", 'ENTEROCOCCUS FAECALIS', 'ENTEROCOCCUS FAECIUM']

for amro in amro_search:
    run_amro_synthetic2(amro)


 10%|█         | 2/20 [00:55<08:24, 28.04s/it] 